# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [60]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import time
import json

# Import API key
from config import g_key
#print(g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [61]:
# Import the clean cities weather data file created in WeatherPy project
cleancitydatapath = "2ex_cleancitydata.csv"

# Read the clean cities weather data file created in WeatherPy project
cities_df = pd.read_csv(cleancitydatapath)
cities_df['Hotel'] = ""
cities_df.head()

,City,Lat,Lng,Temp Max,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,omsukchan,62.53,155.80,-11.88,89.0,0,2.68,RU,1603306757,
1,rikitea,-23.12,-134.97,23.26,66.0,5,3.81,PF,1603306758,
2,sao filipe,14.90,-24.50,28.76,62.0,79,2.19,CV,1603306761,
3,santa isabel do rio negro,-0.41,-65.02,33.77,47.0,88,1.75,BR,1603306762,
4,barrow,71.29,-156.79,1.00,98.0,90,7.20,US,1603306763,


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [62]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [63]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Store "Humidity" as the weight
# HINT: be sure to handle NaN values ... #cities_complete = cities_df.dropna()
weights = cities_df["Humidity"].astype(float)

# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [64]:
idealcities = cities_df.loc[cities_df["Temp Max"] >=18]
#idealcities.head()
evenbettercities = idealcities.loc[idealcities["Temp Max"] <=27]
evenbettercities.to_csv("output_data/V1ex_evenbettervacationspots.csv", index=False, header=True)
evenbettercities.head()

,City,Lat,Lng,Temp Max,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
1,rikitea,-23.12,-134.97,23.26,66.0,5,3.81,PF,1603306758,
10,bunawan,8.17,126.08,22.94,97.0,100,0.78,PH,1603306770,
18,bengkulu,-3.80,102.27,25.23,84.0,100,2.15,ID,1603306782,
21,alofi,-19.06,-169.92,23.00,100.0,100,6.20,NU,1603306790,
22,acajutla,13.59,-89.83,27.00,65.0,20,2.10,SV,1603306496,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [65]:
# Hotel column added to dataframe at start of this script
vacationset = len(evenbettercities)
print(vacationset)

14


In [69]:

# Store latitude and longitude in locations
target_coordinates = evenbettercities[["Lat", "Lng"]]

# set up a parameters dictionary
params = {
    #"location": target_coordinates,
    "keyword": "hotel",
    "radius": 5000,
    "type": "hotel",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
counter = 0

# use iterrows to iterate through pandas dataframe
for index, row in evenbettercities.iterrows():
    
    cityname = row['City']
    
    counter = counter + 1
    
    if counter >= 2:
        print("**Done running the first batch**")
        break
    
    if counter <= 1:
        time.sleep(1)
        print(f"Processing Record {counter} of {vacationset}. City = {cityname}")
        
        # get location from df
        target_coordinates = evenbettercities[["Lat", "Lng"]]
        #print(target_coordinates)
            
        # assemble url and make API request
        #print(f"Retrieving Results for {counter}: {cityname}.")
        
    
        # extract results
        #results = response['results']
    
        try:
            response = requests.get(base_url, params=params).json() 
            print(json.dumps(response, indent=4, sort_keys=True))
            #evenbettercities.loc[index, 'Hotel'] = response[0]['name']                
           
        except (KeyError, IndexError, ValueError):
            print(f"... skipped over  " + cityname + "  because it lacked sufficient information")
            pass

Processing Record 1 of 14. City = rikitea
{
    "html_attributions": [],
    "results": [],
    "status": "INVALID_REQUEST"
}
**Done running the first batch**


In [58]:
evenbettercities.head()

,City,Lat,Lng,Temp Max,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
1,rikitea,-23.12,-134.97,23.26,66.0,5,3.81,PF,1603306758,
10,bunawan,8.17,126.08,22.94,97.0,100,0.78,PH,1603306770,
18,bengkulu,-3.80,102.27,25.23,84.0,100,2.15,ID,1603306782,
21,alofi,-19.06,-169.92,23.00,100.0,100,6.20,NU,1603306790,
22,acajutla,13.59,-89.83,27.00,65.0,20,2.10,SV,1603306496,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
